In [7]:
#
# cs590-chapter4.ipynb - This is a Jupyter Notebook designed to 
# connect to a PostgreSQL database and provide CRUD operations.
#
# This notebook uses a supplemental configuration file that has the
# following format:
# 
# [postgresql]
# host=localhost
# database=Nutrition
# user=cs590
# password=REPLACE_WITH_YOUR_PASSWORD_FOR_THE_DATABASE_USER
#
# We also build a config function to read the data from the
# configuration file to make the code easier to maintain and more
# secure. 
#
# Remember: You should _NEVER_ embed a password in your 
# source code!
#
#------------------------------------------------------------------
# Change History
#------------------------------------------------------------------
# Version   |   Description
#------------------------------------------------------------------
#    1          Initial Development
#------------------------------------------------------------------

import psycopg2
from configparser import ConfigParser

#
# config - This function is designed to read the data from our
# configuration file to allow us to setup the connection to our
# PostgreSQL database.
#
# Please Note: We create this method genericly so that it can
# be reused with other configuration data separated by section
# as key-value pairs
#
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
# 
# Returns:
#   A dictionary with the key-value pairs from the configuration file.
#
def config(filename, section):
    # Initialize a configuration parser
    cp = ConfigParser()
    
    # Acquire data from the configuration file
    cp.read(filename)
    
    # Import configuration section as a dictionary
    block = {}
    
    # Check to make sure that the section exists in our config file
    if cp.has_section(section):
        # Read the entire section
        items = cp.items(section)
        
        # Loop through config items to create the dictionary
        for item in items:
            block[item[0]] = item[1]
    else:
        # If we have gotten here the section did not exist in the
        # config file, so we raise an exception
        raise Exception(f"Section {section} was not found in the {filename} file!")
    
    # Return the dictionary to the calling program
    return block


#
# displayConfig - This function is designed to test the config function.
# This function will call the config function with a filename and a section
# obtain the resulting data, and then print the retrieved information to
# the screen. This can be useful for debugging and testing purposes.
#
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
# 
def displayConfig(filename, section):
    # Obtain configuration data from the config file
    confData = config(filename, section)
    
    # Display results
    print(confData)
    

#
# dbConnect - function designed to connect to the database. This function
# leverages the config method to pull the necessary data from our database
# configuration file.
#
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
# 
# Returns a Connection to the databse or null in the event of a problem
#
def dbConnect(filename, section):
    # Obtain configuration data from the config file
    confData = config(filename, section)

    # create connection variable
    conn = None

    # Create a try/except block for the connection. This is important
    # in order to keep from blowing up the application in the event that
    # we cannot connect to the database.
    try:     
        # Setup the connection to the PostgreSQL server
        conn = psycopg2.connect(
            dbname = confData["database"],
            user = confData["user"],
            password = confData["password"],
            host = confData["host"]
        )

        # Return the database connection for use in the application
        # Don't forget to close the connection when done.
        return conn
    except(Exception, psycopg2.DatabaseError) as error:
        # We had a problem, so lets see what it was
        print(error)
        return None
    
#
# dbTest - a utility method to test connection to the database
# This method will connect to the database with the information from
# the configuration file, validate that the connection was made to the 
# Appropriate database, print the result, and then close the database
# connection. This function is useful for debugging initial connections. 
# 
# Parameters:
#
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def dbTest(filename, section):
    # create our connection
    print('Connecting to our PostgreSQL database...')
    conn = dbConnect(filename, section)

    # Create a try/except block for the connection. This is important
    # in order to keep from blowing up the application in the event that
    # we cannot connect to the database.
    try:     
        # Create a cursor
        cursor = conn.cursor()

        # test the connection
        cursor.execute('SELECT current_database()')

        # display result
        print(cursor.fetchone())

        # close the cursor
        cursor.close()
    except(Exception, psycopg2.DatabaseError) as error:
        # We had a problem, so lets see what it was
        print(error)
    finally:
        # Everything worked well, so we need to close the connection
        if conn is not None:
            conn.close()


#
# create - CRUD Method used to create a new record in a database. 
# This method will be used to create a new record to be added to a database.
# Because PostgreSQL is a relational database, this will add a record to a
# single table.
#
# Parameters:
#     query: SQL statement to insert record into the appropriate database table
#     conn: The database connection to use create the record
#
def createRecord(query, conn):
    cursor = None
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        conn.commit()
        print("Record created successfully.")
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error creating record:", error)
        conn.rollback()
    finally:
        if cursor is not None:
            cursor.close()
                       

#
# testCreateRecord - Method used to test the createRecord function.
# This function will exercise the createRecord function by attempting to add
# a record to the nutdes (Nutrient description) table. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testCreateRecord(filename, section):
    # create our connection
    print('Connecting to our PostgreSQL database...')
    conn = dbConnect(filename, section)

    # Setup the SQL Statement

    query = "INSERT INTO public.nutdes VALUES ( 777, 'Zinc', 'Zn', 'mcg', '05/08/2025', '05/08/2025');"

    createRecord(query,conn)
    conn.close()


#
# readRecord - CRUD Method used to read a record from a database. 
# This method will be used to read a new record from a database table.
# Because PostgreSQL is a relational database, this will read a record from a
# single table.
#
# Parameters:
#     tablename: The table to select the record(s) from
#     clause: the appropriate clause to test for - specifically what you would 
#             see after the SQL WHERE clause
#     conn: The database connection to use create the record
#    
def readRecord(tablename, clause, conn):
    sql = f"SELECT * FROM public.{tablename} WHERE {clause};"
    cursor = None
    try:
        cursor = conn.cursor()
        cursor.execute(sql)
        records = cursor.fetchall()
        return records
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error reading record(s):", error)
        return []
    finally:
        if cursor is not None:
            cursor.close()



#
# testReadRecord - Method used to test the readRecord function.
# This function will exercise the readRecord function by attempting to read
# a record from the nutdes (Nutrient description) table. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testReadRecord(filename, section):
    # create our connection
    print('Connecting to our PostgreSQL database...')
    conn = dbConnect(filename, section)

    # Setup the SQL Statement
    print('Reading Record now...')
    records = readRecord("nutdes",'"Nutrient code" = 777', conn)

    # Display results
    print(records)
    
    conn.close()

#
# updateRecord - CRUD Method used to update a record from a database. 
# This method will be used to update a record from a database table.
# Because PostgreSQL is a relational database, this will update a record from a
# single table.
#
# Parameters:
#     tablename: The table to update the record in
#     clause: The update clause - what you would normally see after the SQL SET clause
#     conn: The database connection to use create the record
#    
def updateRecord(tablename, clause, conn):
    sql = f"UPDATE public.{tablename} SET {clause};"
    cursor = None
    try:
        cursor = conn.cursor()
        cursor.execute(sql)
        conn.commit()
        print('Updating row(s)')
        print("Rows updated:", cursor.rowcount)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error updating record(s):", error)
        conn.rollback()
    finally:
        if cursor is not None:
            cursor.close()



#
# testUpdateRecord - Method used to test the updateRecord function.
# This function will exercise the updateRecord function by attempting to update
# a record from the nutdes (Nutrient description) table. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testUpdateRecord(filename, section):
    # create our connection
    print('Connecting to our PostgreSQL database...')
    conn = dbConnect(filename, section)

    # Setup the SQL Statement
    updateRecord("nutdes",""" "Nutrient description" = 'ZINC' where "Nutrient code" = 777""", conn)    
    conn.close()


#
# deleteRecord - CRUD Method used to delete a record from a database. 
# This method will be used to delete a record from a database table.
# Because PostgreSQL is a relational database, this will update a record from a
# single table.
#
# NOTE: USE WITH CAUTION! THIS METHOD WILL DESTROY DATA
#
# Parameters:
#     tablename: The table to update the record in
#     clause: The update clause - what you would normally see after the SQL WHERE clause
#     conn: The database connection to use create the record
#    
def deleteRecord(tablename, clause, conn):
    sql = f"DELETE FROM public.{tablename} WHERE {clause};"
    cursor = None
    try:
        cursor = conn.cursor()
        cursor.execute(sql)
        conn.commit()
        print('Deleting Row(s)')
        print("Rows deleted:", cursor.rowcount)
        return cursor.rowcount
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error deleting record(s):", error)
        conn.rollback()
        return 0
    finally:
        if cursor is not None:
            cursor.close()



#
# testDeleteRecord - Method used to test the deleteRecord function.
# This function will exercise the deleteRecord function by attempting to remove
# a record from the nutdes (Nutrient description) table. This function will
# only work with the Nutrient database, it is not portable.
#
# Parameters:
#   filename: The name of the file that contains the configuration data
#   section:  The name of the section within the configuration file
#             for which we are retrieving data
#
def testDeleteRecord(filename, section):
    # create our connection
    print('Connecting to our PostgreSQL database...')
    conn = dbConnect(filename, section)

    # Setup the SQL Statement
    records = deleteRecord("nutdes",""" "Nutrient code" = 777 """, conn)    
    conn.close()

if __name__ == '__main__':
    displayConfig('database.conf','postgresql')
    dbTest('database.conf','postgresql')
    testCreateRecord('database.conf','postgresql')
    testReadRecord('database.conf','postgresql')
    testUpdateRecord('database.conf','postgresql')
    testDeleteRecord('database.conf','postgresql')

{'host': 'localhost', 'database': 'calval', 'user': 'postgres', 'password': 'Codio590'}
Connecting to our PostgreSQL database...
('calval',)
Connecting to our PostgreSQL database...
Record created successfully.
Connecting to our PostgreSQL database...
Reading Record now...
[(777, 'Zinc', 'Zn', 'mcg', datetime.date(2025, 5, 8), datetime.date(2025, 5, 8))]
Connecting to our PostgreSQL database...
Updating row(s)
Rows updated: 1
Connecting to our PostgreSQL database...
Deleting Row(s)
Rows deleted: 1
